In [1]:
from __future__ import division
from math import *
import numpy as np


import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams['animation.html'] = 'html5'
%matplotlib inline
from IPython.display import HTML



class PRFModelTrial(object):
    """docstring for PRFModelTrial"""

    def __init__(self, orientation, n_elements, n_samples, sample_duration, bar_width=0.1, ecc_test=1.0):
        super(PRFModelTrial, self).__init__()
        self.orientation = orientation
        self.n_elements = n_elements
        self.n_samples = n_samples
        self.sample_duration = sample_duration
        self.bar_width = bar_width * 2.

        self.rotation_matrix = np.matrix(
            [[cos(self.orientation), -sin(self.orientation)], [sin(self.orientation), cos(self.orientation)]])

        x, y = np.meshgrid(np.linspace(-1, 1, self.n_elements, endpoint = True),
                           np.linspace(-1, 1, self.n_elements, endpoint = True))
        self.xy = np.matrix([x.ravel(), y.ravel()])
        self.rotated_xy = np.array(self.rotation_matrix * self.xy)

        if ecc_test == None:
            self.ecc_test = np.ones_like(x.ravel(), dtype=bool)
        else:
            self.ecc_test = np.sqrt((np.array(self.xy) ** 2).sum(axis=0)) <= ecc_test

    def in_bar(self, time=0):
        """in_bar, a method, not Ralph."""
        # a bar of self.bar_width width
        position = 2.0 * ((time * (1.0 + self.bar_width / 2.0)
                           ) - (0.5 + self.bar_width / 4.0))
        # position = 2.0 * ((time * (1.0 + self.bar_width)) - (0.5 + self.bar_width / 2.0))
        extent = [-self.bar_width / 2.0 + position,
                  self.bar_width / 2.0 + position]
        # rotating the xy matrix itself allows us to test only the x component
        return ((self.rotated_xy[0, :] >= extent[0]) * (self.rotated_xy[0, :] <= extent[1])* self.ecc_test).reshape((self.n_elements, self.n_elements))
        # return ((self.rotated_xy[0,:] >= extent[0]) * (self.rotated_xy[0,:] <= extent[1])).reshape((self.n_elements, self.n_elements))

    def pass_through(self):
        """pass_through models a single pass-through of the bar, 
        with padding as in the padding list for start and end."""

        self.pass_matrix = np.array(
            [self.in_bar(i) for i in np.linspace(0.0, 1.0, self.n_samples, endpoint=True)])


def create_visual_designmatrix_all(
        bar_dur_in_TR=32,
        iti_duration=1,
        bar_width=0.1,
        n_pixels=100,
        thetas=[-1, 0, -1, 45, 270, -1,  315,
                180, -1,  135,   90, -1,  225, -1],
        nr_timepoints=462):
    ITIs = np.zeros((iti_duration, n_pixels, n_pixels))
    all_bars = []
    for x in thetas:
        all_bars.extend(ITIs)
        if x == -1:
            all_bars.extend(np.zeros((bar_dur_in_TR, n_pixels, n_pixels)))
        else:
            pmt = PRFModelTrial(orientation=-np.radians(x) - np.pi / 2.0, n_elements=n_pixels,
                                n_samples=bar_dur_in_TR + 1, sample_duration=1, bar_width=bar_width)
            pmt.pass_through()
            pmt.pass_matrix = pmt.pass_matrix[:-1]
            all_bars.extend(pmt.pass_matrix)

    # swap axes for popeye:
    visual_dm = np.transpose(np.array(all_bars), [1, 2, 0])
    visual_dm = np.round(visual_dm[:, :, :nr_timepoints]).astype(np.int16)

    return visual_dm

fig = plt.figure()
fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)
plt.axis('off')

# run the code
print('computating stimuli...')
thetas = [-1, 0, -1, 45, 270, -1,  315, 180, -1,  135,   90, -1,  225, -1]
visual_dm = create_visual_designmatrix_all(thetas = thetas) 
shape_visual_dm = visual_dm.shape
ims = []
for t in range(shape_visual_dm[2]):
    im = plt.imshow(visual_dm[:,:,t],animated=True,cmap='viridis',clim=[0,1])
    plt.axis('off')
    ims.append([im])
plt.close()

print('computation done')
speed = 10
print('preparing animation, speed = %1.1fx'%speed)
TR = 945
ani = animation.ArtistAnimation(fig, ims,interval=TR/speed, blit=True)
print('saving video...')
ani.save('out2.mp4')
print('done')


computating stimuli...
computation done
preparing animation, speed = 10.0x
saving video...
done


In [2]:
ani